### Импорты

In [1]:
import sqlite3
import pandas as pd
import nltk
nltk.download('framenet_v17')
from nltk.corpus import framenet as fn
import re
from string import punctuation

[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/annsmklv/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


### База данных

In [2]:
con = sqlite3.connect('MultilingualFramenet.db')  
cur = con.cursor()  

### Berkeley FrameNet

Данные для BFN будем брать из nltk

#### 1. Фреймы

In [14]:
data = []
for frame in fn.frames():
    data.append((int(frame.ID), frame.name, " ".join([fe for fe in frame.FE]), frame.definition, f"https://framenet.icsi.berkeley.edu/fnReports/data/frameIndex.xml?frame={frame.name}"))

In [15]:
example = data[:3]
bfn_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
bfn_frames


,id,frame,frame_elements,definition,link
0,2031,Abandonment,Agent Theme Place Time Manner Duration Explana...,An Agent leaves behind a Theme effectively ren...,https://framenet.icsi.berkeley.edu/fnReports/d...
1,262,Abounding_with,Theme Location Degree Depictive Time,A Location is filled or covered with the Theme...,https://framenet.icsi.berkeley.edu/fnReports/d...
2,830,Absorb_heat,Entity Container Heat_source Place Medium Mann...,An Entity (generally food) is exposed to a Hea...,https://framenet.icsi.berkeley.edu/fnReports/d...


Загружаем в таблицу

In [16]:
cur.execute("""
CREATE TABLE bfn (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO bfn VALUES (?, ?, ?, ?, ?)", data)
con.commit()

#### 2. Лексические единицы

In [148]:
lexical_units = []
for frame in fn.frames():
    frame_id = frame.ID
    lang = 'en'
    lus = frame.lexUnit.items()
    for k, v in lus:
        id = v.ID
        word, pos = k.split('.')
        lexical_units.append((id, word, pos, lang, frame_id))

In [150]:
example_lus = pd.DataFrame({'id': [x[0] for x in lexical_units],
                           'word':[x[1] for x in lexical_units],
                           'pos':[x[2] for x in lexical_units],
                           'lang': [x[3] for x in lexical_units],
                           'frame_id':[x[4] for x in lexical_units]}, index=None)
example_lus


,id,word,pos,lang,frame_id
0,14839,abandon,v,en,2031
1,14841,leave,v,en,2031
2,14842,abandonment,n,en,2031
3,14843,abandoned,a,en,2031
4,15317,forget,v,en,2031
...,...,...,...,...,...
13567,14003,plug away,v,en,1892
13568,14005,bang away,v,en,1892
13569,16474,man,v,en,2603
13570,16475,staff,v,en,2603


Создаем таблицу и добавляем туда наши данные

In [152]:
cur.execute("""
CREATE TABLE bfn_lu (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO bfn_lu VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

#### 3. Типы отношений и отношения между фреймами

In [130]:
types = []
for type in list(fn.frame_relation_types()):
    types.append((int(type.ID), type.name, type.superFrameName, type.subFrameName, ''))

In [131]:
example = types[:3]
example_types = pd.DataFrame({'id': [x[0] for x in example],
                           'name':[x[1] for x in example],
                           'superframe':[x[2] for x in example],
                           'subframe':[x[3] for x in example]}, index=None)
example_types

,id,name,superframe,subframe
0,10,Causative_of,Causative,Inchoative/state
1,9,Inchoative_of,Inchoative,Stative
2,1,Inheritance,Parent,Child


In [ ]:
cur.execute("""
CREATE TABLE relation_types (
    id INT, 
    type TEXT,
    superframe TEXT,
    subframe TEXT,
    definition TEXT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO relation_types VALUES (?, ?, ?, ?, ?)", types)
con.commit()

Так как соединяем по id, создадим словарь, в котором ключами будут названия фреймов, а значениями - их id

In [134]:
ids = {}
for t in types:
    ids[t[1]] = t[0]

In [135]:
bfn_relations = []
for rel in fn.frame_relations():
    bfn_relations.append((rel.superFrame.ID, rel.subFrame.ID, ids[rel.type.name]))

In [136]:
example = bfn_relations[:3]
example_rel = pd.DataFrame({'parent_id': [x[0] for x in example],
                           'child_id':[x[1] for x in example],
                           'relation_id':[x[2] for x in example]}, index=None)
example_rel

,parent_id,child_id,relation_id
0,262,1904,1
1,1602,1603,1
2,124,236,1


In [137]:
cur.execute("""
CREATE TABLE bfn_frame_relations (
    parent_id INT, 
    child_id INT, 
    relation_id TEXT
)
""")
cur.executemany("INSERT INTO bfn_frame_relations VALUES (?, ?, ?)", bfn_relations)
con.commit()

### Ресурсы

Информация была собрана нами вручную.

In [17]:
meta = [
    ('DiCoEnviro', 'Экология', 'Французский, английский, португальский, китайский',	'https://olst.ling.umontreal.ca/dicoenviro/framed/index.php', '-', '', 203, None, ''),
    ('DiCoInfo', 'Информация, интернет', 'Французский, английский, арабский', 'https://olst.ling.umontreal.ca/dicoinfo/framed/', '-', '', None, None, 'https://olst.ling.umontreal.ca/dicoinfo/framed/'),
    ('SweFN', 'Без определённой тематики', 'Шведский', 'https://spraakbanken.gu.se/karp/#?mode=DEFAULT&resources=swefn&lang=eng&advanced=false', 'https://spraakbanken.gu.se/en/resources?s=SweFN&language=All', '', 1195, 39210, 'https://spraakbanken.gu.se/en/resources/swefn'),
    ('FrameNet Brasil', 'Без определённой тематики', 'Бразильский португальский, английский (вероятно, ещё)', 'https://webtool.framenetbr.ufjf.br/index.php/webtool/report/frame/main', '-', '', None, None, ''),
    ('German FrameNet', 'Без определённой тематики', 'Немецкий', 'https://framenet-constructicon.hhu.de/framenet/', 'https://www.coli.uni-saarland.de/projects/salsa/corpus/', '', 1285, 13905, 'https://framenet-constructicon.hhu.de/project/publications'),
    ('Spanish FrameNet', 'Без определённой тематики', 'Испанский', 'http://gemini.uab.es/SFN', '-', '', None, None, ''),
    ('GFOL', 'Без определённой тематики', 'Немецкий', 'https://coerll.utexas.edu/frames/frame-index', '-', '', 29, None, '')
]								

In [18]:
example = meta[:5]
example_meta = pd.DataFrame({'resource': [x[0] for x in example],
                           'theme':[x[1] for x in example],
                           'language':[x[2] for x in example],
                           'online':[x[3] for x in example],
                           'offline':[x[4] for x in example],
                           'library':[x[5] for x in example],
                           'size':[x[6] for x in example],
                           'number_of_LUs':[x[7] for x in example],
                           'publications':[x[8] for x in example]
                           }, index=None)
example_meta

,resource,theme,language,online,offline,library,size,number_of_LUs,publications
0,DiCoEnviro,Экология,"Французский, английский, португальский, китайский",https://olst.ling.umontreal.ca/dicoenviro/fram...,-,,203.0,NaN,
1,DiCoInfo,"Информация, интернет","Французский, английский, арабский",https://olst.ling.umontreal.ca/dicoinfo/framed/,-,,NaN,NaN,https://olst.ling.umontreal.ca/dicoinfo/framed/
2,SweFN,Без определённой тематики,Шведский,https://spraakbanken.gu.se/karp/#?mode=DEFAULT...,https://spraakbanken.gu.se/en/resources?s=SweF...,,1195.0,39210.0,https://spraakbanken.gu.se/en/resources/swefn
3,FrameNet Brasil,Без определённой тематики,"Бразильский португальский, английский (вероятн...",https://webtool.framenetbr.ufjf.br/index.php/w...,-,,NaN,NaN,
4,German FrameNet,Без определённой тематики,Немецкий,https://framenet-constructicon.hhu.de/framenet/,https://www.coli.uni-saarland.de/projects/sals...,,1285.0,13905.0,https://framenet-constructicon.hhu.de/project/...


In [19]:
cur.execute("""
CREATE TABLE resources (
    resource TEXT,
    theme TEXT,
    language TEXT, 
    online_availability TEXT,
    offline_availability TEXT,
    library TEXT,
    size INT,
    number_of_LUs INT,
    publication TEXT
)
""")
cur.executemany("INSERT INTO resources VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", meta)
con.commit()

### SweFN


Данные из SweFN будем доставать парсингом xml-файла

#### 1. Фреймы

In [153]:
import xml.etree.ElementTree as ET
tree = ET.parse('./swefn-full/swefn.xml')
root = tree.getroot()
lexical_entries = root.findall('Lexicon/LexicalEntry/Sense')

In [21]:
data = []
for i, le in enumerate(lexical_entries):
    id = i
    frame = le.attrib['id'].split('--')[1]
    frame_elements = []
    definition = ''
    attrs = [x.attrib for x in le.findall('feat')]
    for attr in attrs:
        if attr['att'] == 'coreElement' or attr['att'] == 'peripheralElement':
            frame_elements.append(attr['val'])
        elif attr['att'] == 'definition':
            definition = re.sub('<.*?>', '', attr['val'])
    data.append((i, frame, " ".join(frame_elements), definition, ''))

In [23]:
cur.execute("""
CREATE TABLE swefn (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO swefn VALUES (?, ?, ?, ?, ?)", data)
con.commit()

#### 2. Связи между фреймами в BFN и фреймами в SweFN

In [102]:
import xml.etree.ElementTree as ET
tree = ET.parse('./swefn-full/swefn.xml')
root = tree.getroot()
lexical_entries = root.findall('Lexicon/LexicalEntry/Sense')

Большая часть фреймов в шведском фреймнете являются копией фреймов из BFN, поэтому мы вытаскиваем аттрибут BFNID. Для фреймов, для которых нет такого аттрибута, мы будем проставлять связи вручную

In [103]:
collocations = []
for i, le in enumerate(lexical_entries):
    frame = le.attrib['id'].split('--')[1]
    feats = le.findall('feat')
    if_colloc = False
    for f in feats:
        if f.attrib['att'] == "BFNID":
            collocations.append((frame, f.attrib['val']))
            if_colloc = True
    if not if_colloc:
        collocations.append((frame, None))

In [104]:
query = """
SELECT id, frame
FROM bfn
"""
cur.execute(query)
results = cur.fetchall()

In [109]:
collocations[0]

('Event_in_history', 'Event_in_history')

In [111]:
bfn = {}
for r in results:
    bfn[r[1]] = r[0]
    
bfn_swefn = []
for i, frame in enumerate(collocations):
    if frame[1] in bfn:
        bfn_swefn.append((frame[1], i, frame[0], "ExactMatch"))
    else:
        bfn_swefn.append((None, i, frame[0], "??"))

In [20]:
cur.execute("""
CREATE TABLE bfn_swefn_match (
    bfn_frame_id INT, 
    swefn_frame_id INT, 
    match TEXT
)
""")
cur.executemany("INSERT INTO bfn_swefn_match VALUES (?, ?, ?)", bfn_swefn)
con.commit()

#### 3. SweFN lexical units

In [320]:
import xml.etree.ElementTree as ET
tree = ET.parse('./swefn-full/swefn-ex.xml')
root = tree.getroot()
tokens = root.findall('corpus/text/sentence/token')

In [322]:
query = """
SELECT id, frame
FROM swefn
"""
cur.execute(query)
results = cur.fetchall()

In [167]:
results_dict = {}
for l in results:
    results_dict[l[1]] = l[0]

In [336]:
pos_list = {'AB': 'adv', 
       'JJ': 'a', 
       'NN': 'n', 
       'VB': 'v'
}

lang = 'sv'
lus = []
used_lus = {}
id = 1
for token in tokens:
    token_dict = token.attrib
    if token_dict['swefn'] == '|':
        continue
    frame = token_dict['swefn'].replace('|', '')
    word = token_dict['lemma'].replace('|', '')
    pos = token_dict['pos']

    if word in used_lus:
        stop = False
        for info in used_lus[word]:
            if info[0] == frame:
                if info[1] == pos:
                    stop = True
        if stop:
            continue
        else:
            used_lus[word].append((frame, pos))
            if frame in results_dict:
                lus.append((id, word, pos_list[pos], lang, results_dict[frame]))
                id += 1 
    else:
        used_lus[word] = [(frame, pos)]
        if frame in results_dict:
                lus.append((id, word, pos_list[pos], lang, results_dict[frame]))
                id += 1 

In [338]:
cur.execute("""
CREATE TABLE swefn_lu (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO swefn_lu VALUES (?, ?, ?, ?, ?)", lus)
con.commit()

### DiCoEnviro

In [181]:
data = pd.read_csv('./data/dicoenviro.csv').to_numpy()

#### 1. Фреймы

In [34]:
frames = []
for i, frame in enumerate(data):
    frames.append((i, frame[1], frame[7], frame[9], frame[8]))

In [35]:
example = frames[:3]
dicoenviro_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
dicoenviro_frames

,id,frame,frame_elements,definition,link
0,0,Abundance,Theme Location,The Theme is present in a Location in a specif...,https://olst.ling.umontreal.ca/dicoenviro/fram...
1,1,Accumulating,Patient Location,A Patient becomes increasingly present in a Lo...,https://olst.ling.umontreal.ca/dicoenviro/fram...
2,2,Adapting,Patient Cause,A Patient evolves in order to deal with a Cause.,https://olst.ling.umontreal.ca/dicoenviro/fram...


In [36]:
cur.execute("""
CREATE TABLE dicoenviro (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO dicoenviro VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. Лексические единицы

In [222]:
id = 0
lexical_units = []
for i, frame in enumerate(data):
    englishlus = frame[2]
    frenchlus =	frame[3]
    portugueselus =	frame[4]
    chineselus	= frame[5]
    spanishlus =  frame[6]
    pos = ''
    
    if englishlus and isinstance(englishlus, str): 
        lang = 'en'
        lus = englishlus.split('\n')
        for lu in lus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            if lu[-1].isdigit():
                lu = lu[:-1]
            elif lu[-2].isdigit():
                lu = lu[:-2]
            lexical_units.append((id, lu, pos, lang, i))
            id += 1
    
    if frenchlus and isinstance(frenchlus, str):
        lang = 'fr'
        lus = frenchlus.split('\n')
        for lu in lus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            if lu[-1].isdigit():
                lu = lu[:-1]
            elif lu[-2].isdigit():
                lu = lu[:-2]
            lexical_units.append((id, lu, pos, lang, i))
            id += 1

    if isinstance(portugueselus, str) and portugueselus:
        lang = 'pt'
        lus = portugueselus.split('\n')
        for lu in lus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            if lu[-1].isdigit():
                lu = lu[:-1]
            elif lu[-2].isdigit():
                lu = lu[:-2]
            lexical_units.append((id, lu, pos, lang, i))
            id += 1

    if chineselus and isinstance(chineselus, str):
        lang = 'zh'
        lus = chineselus.split('\n')
        for lu in lus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            if lu[-1].isdigit():
                lu = lu[:-1]
            elif lu[-2].isdigit():
                lu = lu[:-2]
            lexical_units.append((id, lu, pos, lang, i))
            id += 1
    
    if spanishlus and isinstance(spanishlus, str):
        lang = 'es'
        lus = spanishlus.split('\n')
        for lu in lus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            if lu[-1].isdigit():
                lu = lu[:-1]
            elif lu[-2].isdigit():
                lu = lu[:-2]
            lexical_units.append((id, lu, pos, lang, i))
            id += 1

In [ ]:
cur.execute("""
CREATE TABLE dicoenviro_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO dicoenviro_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### GFOL

In [226]:
data = pd.read_csv('./data/gfol.csv').to_numpy()

#### 1. Frames

In [72]:
frames = []
for i, frame in enumerate(data):
    frame_name = frame[0]
    fes = frame[2].replace('\n', ' ')
    definition = frame[1]
    link = frame[4]
    frames.append((i, frame_name, fes, definition, link))

In [74]:
example = frames[:3]
gfol_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
gfol_frames

,id,frame,frame_elements,definition,link
0,0,Age - TBP,Entity Age Expressor,The words and phrases in this frame reference ...,https://frames.coerll.utexas.edu/frames/age_TBP
1,1,Arguing,Side_1 Side_2 Sides Issue,The words and phrases in this frame describe a...,https://frames.coerll.utexas.edu/frames/arguing
2,2,Body Descriptions,Body_part Individual Descriptor,This frame is evoked by words describing the e...,https://frames.coerll.utexas.edu/frames/body%2...


In [75]:
cur.execute("""
CREATE TABLE gfol (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO gfol VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. Lexical units

In [235]:
pos_list = {'adjective' : 'a',
 'adjective;' : 'a',
 'adverb' : 'adv',
 'conjunction' : 'c',
 'construction' : 'idio',
 'multi-word expression' : 'idio',
 'noun' : 'noun',
 'noun;' : 'noun',
 'prefix' : 'prefix',
 'preposition' : 'prep',
 'verb' : 'v',
 'verb;' : 'v'}

id = 0
lexical_units = []
lang = 'de'
for i, frame in enumerate(data):
    lus_list = frame[3].split('\n')
    lus_list = [x.strip() for x in lus_list]
    for lu in lus_list: 
        lu = lu.split(' - ')
        if len(lu) == 1 or lu[1].isupper():
            continue
        lu, pos = lu[0], lu[1]
        lexical_units.append((id, lu, pos_list[pos], lang, i))
        id += 1

In [237]:
cur.execute("""
CREATE TABLE gfol_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO gfol_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### DiCoInfo

In [238]:
data = pd.read_csv('./data/dicoinfo.csv').to_numpy()

#### 1. Frames

In [63]:
frames = []
for i, frame in enumerate(data):
    frame_name = frame[1]
    fes = frame[5]
    link = frame[6]
    definition = frame[7]
    frames.append((i, frame_name, fes, definition, link))

In [64]:
example = frames[:3]
dicoinfo_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
dicoinfo_frames

,id,frame,frame_elements,definition,link
0,0,Access_scenario,Agent Location Destination,Agent (user or system) connects to Location (n...,https://olst.ling.umontreal.ca/dicoinfo/framed...
1,1,Activity_resume,Patient Agent,Agent (user) resumes activity of Patient (prog...,https://olst.ling.umontreal.ca/dicoinfo/framed...
2,2,Activity_scenario,Agent Patient Environment1 Environment2,Agent (user) or Instrument (système or compute...,https://olst.ling.umontreal.ca/dicoinfo/framed...


In [65]:
cur.execute("""
CREATE TABLE dicoinfo (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO dicoinfo VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. LUs

In [274]:
lexical_units = []
id = 0
for i, frame in enumerate(data):
    englishlus = frame[2]
    frenchlus = frame[3]
    arabiclus = frame[4]

    if englishlus and isinstance(englishlus, str):
        lang = 'en'
        englishlus = englishlus.split('\n')
        for lu in englishlus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            lu_list = lu.split('.')
            if len(lu_list) == 2:
                lu = lu_list[0]
            elif len(lu_list) > 2:
                lu_list = lu_list[:-1]
                lu = ".".join(lu_list)
            else:
                lu = lu
            lexical_units.append((id, lu, '', lang, i))
            id += 1
    
    if frenchlus and isinstance(frenchlus, str):
        lang = 'fr'
        frenchlus = frenchlus.split('\n')
        for lu in frenchlus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            lu_list = lu.split('.')
            if len(lu_list) == 2:
                lu = lu_list[0]
            elif len(lu_list) > 2:
                lu_list = lu_list[:-1]
                lu = ".".join(lu_list)
            else:
                lu = lu
            lexical_units.append((id, lu, '', lang, i))
            id += 1
    
    if arabiclus and isinstance(arabiclus, str):
        lang = 'ar'
        arabiclus = arabiclus.split('\n')
        for lu in arabiclus:
            lu = lu.replace(',', '').replace(';', '')
            if lu == "":
                continue
            lu_list = lu.split('.')
            if len(lu_list) == 2:
                lu = lu_list[0]
            else:
                continue
            lexical_units.append((id, lu, '', lang, i))
            id += 1

In [276]:
cur.execute("""
CREATE TABLE dicoinfo_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO dicoinfo_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### German FrameNet

In [277]:
data = pd.read_csv('./data/germanframenet.csv').to_numpy()

#### 1. Frames

In [86]:
frames = []
for frame in data:
    id = frame[0]
    frame_name = frame[1]
    definition = frame[4]
    fes_list = frame[3]
    fes = fes_list.translate(str.maketrans('', '', punctuation))
    link = frame[6]
    frames.append((id, frame_name, fes, definition, link))

In [87]:
example = frames[:3]
germanfn_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
germanfn_frames

,id,frame,frame_elements,definition,link
0,407,Abbrechen_von,Ganzes Teil,Ein einzelnes FE:TeilTeil bricht von einem F...,https://framenet-constructicon.hhu.de/framenet...
1,230,Abfangen,Abfänger Objekt,Ein willentlich handelnder FE:AbfängerAbfänge...,https://framenet-constructicon.hhu.de/framenet...
2,772,Abfeuerungsort,Auslöseort,Ein FE:AuslösenderAuslösender aktiviert von e...,https://framenet-constructicon.hhu.de/framenet...


In [88]:
cur.execute("""
CREATE TABLE germanfn (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO germanfn VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. Frames

In [284]:
pos_list = {'ADJ' : 'a', 
            'ADV' : 'adv', 
            'KONJ' : 'c', 
            'MWA' : 'MWA', 
            'N' : 'n', 
            'NONE' : '', 
            'PRAEP' : 'prep', 
            'PRO' : 'PRO', 
            'PTK' : 'PTK', 
            'V' : 'v'}

lexical_units = []
id = 0
lang = "de"
for frame in data:
    frame_id = frame[0]
    units = frame[2]
    if units == "[]":
        continue
    lus = units.split('),')
    for lu_pos in lus:
        lu_pos = lu_pos.translate(str.maketrans('', '', punctuation)).strip()
        lu_pos = lu_pos.split(' ')
        if len(lu_pos) == 2:
            lu, pos = lu_pos
        else:
            pos = lu_pos[-1]
            lu = ' '.join(lu_pos[:-1])
        lexical_units.append((id, lu, pos_list[pos], lang, frame_id))
        id += 1

In [286]:
cur.execute("""
CREATE TABLE germanfn_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO germanfn_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### Spanish FrameNet

In [287]:
data = pd.read_csv('./data/spanishframenet.csv').to_numpy()

#### 1. Frames

In [97]:
frames = []
for i, frame in enumerate(data):
    frame_name = frame[1]
    fes = frame[3].replace('\n','')
    frames.append((i, frame_name, fes, frame[5], frame[4]))

In [98]:
example = frames[:3]
spanishfn_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
spanishfn_frames

,id,frame,frame_elements,definition,link
0,0,Abounding_with,Location Theme,A Location is filled or covered with the Theme...,https://sfn.spanishfn.org/FRAMES/frames/Abound...
1,1,Achieving_first,Cognizer New_idea,A Cognizer introduces a New_idea into society....,https://sfn.spanishfn.org/FRAMES/frames/Achiev...
2,2,Activity,Activity Agent,This is an abstract frame for durative activit...,https://sfn.spanishfn.org/FRAMES/frames/Activi...


In [99]:
cur.execute("""
CREATE TABLE spanishfn (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO spanishfn VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. LUs

In [305]:
lexical_units = []
id = 0
lang = 'es'
for i, frame in enumerate(data):
    units = frame[2]
    if units and isinstance(units, str):
        lus_list = units.split(', ')
        for lu_pos in lus_list:
            lu_pos.replace('\n', '')
            lu, pos = lu_pos.split('.')
            pos = pos.replace('\n', '')
            lexical_units.append((id, lu, pos, lang, i))
            id += 1

In [307]:
cur.execute("""
CREATE TABLE spanishfn_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO spanishfn_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### Brasil FN

In [308]:
data = pd.read_csv('./data/brasilframenet.csv').to_numpy()

#### 1. Frames

In [51]:
frames = []
for frame in data:
    id = frame[0]
    frame_name = frame[1]
    fes = frame[6]
    if type(fes) is not float:
        fes = fes.replace('\n', ' ')
    definition = frame[7]
    link = frame[9]
    frames.append((id, frame_name, fes, definition, link))

In [52]:
example = frames[:3]
brasilfn_frames = pd.DataFrame({'id': [x[0] for x in example],
                           'frame':[x[1] for x in example],
                           'frame_elements':[x[2] for x in example],
                           'definition':[x[3] for x in example],
                           'link': [x[4] for x in example]}, index=None)
brasilfn_frames

,id,frame,frame_elements,definition,link
0,908,Abandonment,Agent Theme,An Agent leaves behind a Theme effectively ren...,https://webtool.framenetbr.ufjf.br/index.php/w...
1,237,Abounding_with,Location Theme,A Location is filled or covered with the Theme...,https://webtool.framenetbr.ufjf.br/index.php/w...
2,469,Absorb_heat,Container Entity Heat_source,An Entity (generally food) is exposed to a Hea...,https://webtool.framenetbr.ufjf.br/index.php/w...


In [53]:
cur.execute("""
CREATE TABLE brasilfn (
    id INT, 
    frame TEXT, 
    frame_elements TEXT,
    definition TEXT,
    link TEXT,
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO brasilfn VALUES (?, ?, ?, ?, ?)", frames)
con.commit()

#### 2. LUs

In [317]:
def extract_lus(lang, lus, id, frame_id):
    id = id
    lex_un = []
    lus = lus.split('\n')
    for lu in lus:
        lu = lu.replace(';', '').replace(',', '')
        if lu == "":
            continue
        lu_list = lu.split('.')
        if len(lu_list) == 2:
            lu, pos = lu_list
        elif len(lu_list) > 2:
            pos = lu_list[-1]
            lu = ".".join(lu_list[:-1])
        else:
            continue
        lex_un.append((id, lu, pos, lang, frame_id))
        id += 1
    return lex_un, id

lexical_units = []
id = 0
for frame in data:
    frame_id = frame[0]
    spanishlus = frame[2]
    portugueselus = frame[3]
    frenchlus = frame[4]
    englishlus = frame[5]

    if spanishlus and isinstance(spanishlus, str):
        lex_un, id = extract_lus("es", spanishlus, id, frame_id)
        lexical_units.extend(lex_un)
    
    if portugueselus and isinstance(portugueselus, str):
        lex_un, id = extract_lus("pt", portugueselus, id, frame_id)
        lexical_units.extend(lex_un)
    
    if frenchlus and isinstance(frenchlus, str):
        lex_un, id = extract_lus("fr", frenchlus, id, frame_id)
        lexical_units.extend(lex_un)
    
    if englishlus and isinstance(englishlus, str):
        lex_un, id = extract_lus("en", englishlus, id, frame_id)
        lexical_units.extend(lex_un)

In [319]:
cur.execute("""
CREATE TABLE brasilfn_lus (
    id INT, 
    word TEXT,
    pos TEXT,
    lang TEXT,
    frame_id INT, 
    PRIMARY KEY (id)
)
""")
cur.executemany("INSERT INTO brasilfn_lus VALUES (?, ?, ?, ?, ?)", lexical_units)
con.commit()

### Multilingual FN

In [115]:
data = pd.read_csv('./data/MultilingualFN.csv').to_numpy()

In [123]:
frames = []
for d in data:
    frames.append((d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8], d[9],
                  d[10], d[11], d[12], d[13], d[14], d[15], d[16], d[17]))

In [124]:
cur.execute("""
CREATE TABLE multilingualfn (
    multilingual_id INT, 
    frame_name TEXT,
    swefn_frame_id INT,
    swefn_match_evaluation TEXT,
    brasilfn_frame_id INT,
    brasilfn_match_evaluation TEXT,
    dicoenviro_frame_id INT,
    dicoenviro_match_evaluation TEXT,
    bfn_frame_id INT,
    bfn_match_evaluation TEXT,
    gfol_frame_id INT,
    gfol_match_evaluation TEXT,
    dicoinfo_frame_id INT,
    dicoinfo_match_evaluation TEXT,
    germanfn_frame_id INT,
    germanfn_match_evaluation TEXT,
    spanishfn_frame_id INT,
    spanishfn_match_evaluation TEXT,
    PRIMARY KEY (multilingual_id)
)
""")
cur.executemany("INSERT INTO multilingualfn VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", frames)
con.commit()

### Multilingual LUs

In [3]:
data = pd.read_csv('MultilingualLUs.csv').to_numpy()

In [4]:
frames = []
for id, d in enumerate(data):
    frames.append((id + 1, d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8], d[9]))

In [7]:
cur.execute("""
CREATE TABLE multilingualfn_lus (
    lu_id INT, 
    lu TEXT,
    language TEXT, 
    swefn_lu_id TEXT, 
    gfol_lu_id TEXT, 
    dicoenviro_lu_id TEXT, 
    dicoinfo_lu_id TEXT,
    germanfn_lu_id TEXT,
    brasilfn_lu_id TEXT, 
    spanishfn_lu_id TEXT,
    bfn_lu_id TEXT,
    PRIMARY KEY (lu_id)
)
""")
cur.executemany("INSERT INTO multilingualfn_lus VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", frames)
con.commit()